In [209]:
import numpy as np
import os
import pandas as pd
import json

from tqdm.notebook import tqdm

In [11]:
dir = '/content/drive/My Drive/Colab Notebooks/Genshin Impact Monte Carlo/'
os.listdir(dir)

['genshin_montecarlo.ipynb', 'banners.json']

In [55]:
# Load banner data from JSON
jsonpath = dir+'banners.json'

with open(jsonpath, 'r') as j:
  all_banners = json.loads(j.read())

In [43]:
print(all_banners)

{'banner': [{'geninfo': {'name': 'Venti', 'prob5': 0.006, 'prob4': 0.051, 'prob3': 0.943, '5startotal': 6, '4startotal': 29}, '5stardrop_char': ['Venti', 'Keqing', 'Mona', 'Qiqi', 'Diluc', 'Jean'], '4stardrop_char': ['Barbara', 'Fischl', 'Xiangling', 'Sucrose', 'Chongyun', 'Noelle', 'Bennett', 'Ninguang', 'Xingqiu', 'Beidou', 'Razor'], 'rateups': {'5star': [1], '4star': [1, 2, 3]}, 'rateup_perc': {'5star': 0.5, '4star': 0.5}, 'pitypullnum': {'5star': 90, '4star': 10}}]}


In [56]:
# Define the banner class based on JSON data
class Banner():
  def __init__(self, banner):

    self.list_chars = {}

    self.name = banner['geninfo']['name']
    self.prob5 = banner['geninfo']['prob5']
    self.prob4 = banner['geninfo']['prob4']
    self.prob3 = banner['geninfo']['prob3']
    self.total5s = banner['geninfo']['5startotal']
    self.total4s = banner['geninfo']['4startotal']

    self.list_chars['5 star'] = banner['5stardrop_char']
    self.list_chars['4 star'] = banner['4stardrop_char']

    self.rateups = banner['rateups']
    self.rateups_perc = banner['rateup_perc']

    self.pitynum = banner["pitypullnum"]

    self.startup_text()

  def startup_text(self):
    print("Loaded Banner: " + self.name)

    for stars in self.rateups:
      print("Characters on " + stars + " rateup: " + str([self.list_chars[stars][x] for x in self.rateups[stars]]) + " at " + str(self.rateups_perc[stars]*100) + "%")

In [205]:
# Define simulator

class MonteCarloSimulator():
  def __init__(self, Banner, timesrolled=0, nrolls=5, nsims=5):
    self.Banner = Banner
    self.name = self.Banner.name
    self.nrolls = nrolls
    self.nsims = nsims
    self.timesrolled = timesrolled # Number of times already rolled on the banner
    
    self.get4rated = False
    self.get5rated = False

    self.simresults = []

  def _roll1(self)->str:
    roll = ""
    thresh5 = self.Banner.prob5*1000
    thresh4 = self.Banner.prob4*1000
    whichchar = np.random.randint(0, 1000)
    if whichchar <= thresh5:
      roll = self._rollforstar('5 star')
    elif whichchar <= thresh4:
      roll = self._rollforstar('4 star')
    else:
      roll = '3 star'
    return roll

    

  def _rollforstar(self, star)->str:
    # roll = -1

    if self.get5rated:
      whichchar = np.random.randint(0, len(self.Banner.rateups['5 star']))
      self.get5rated = False
    elif self.get4rated:
      whichchar = np.random.randint(0, len(self.Banner.rateups['4 star']))
      self.get4rated = False

    israted = np.random.randint(0, 1000)
    # print(israted)
    if israted < self.Banner.rateups_perc[star]*1000:
      # print('Get rated')
      # Get a rated up char!
      whichchar = np.random.randint(0, len(self.Banner.rateups[star]))
      # rolledchar = self.Banner.total5s[whichchar]
    else:
      # Get an unrated-up char!
      # print('Get unrated')
      whichchar = np.random.randint(0, len(self.Banner.list_chars[star]))
      # print(whichchar)
      while whichchar in self.Banner.rateups[star]:
        whichchar = np.random.randint(0, len(self.Banner.list_chars[star]))
      
    return self.Banner.list_chars[star][whichchar]
      



  def _sim1(self)->dict:

    results = {}

    simrolled = self.timesrolled
    if simrolled>0 and (simrolled % self.Banner.pitynum['5 star'] == 0):
      guarantee5 = True
    if simrolled>0 and (simrolled % self.Banner.pitynum['4 star'] == 0):
      guarantee4 = True
    else:
      guarantee5, guarantee4 = False, False
    
    for i in range(self.nrolls):
      roll = ""
      if guarantee5:
        roll = self._rollforstar('5 star')
        guarantee5 = False
      elif guarantee4:
        roll = self._rollforstar('4 star')
        guarantee4 = False
      else:        
        roll = self._roll1()


      # print(roll)

      if roll not in results.keys():
        results[roll] = 1
        if roll in self.Banner.list_chars['5 star'] and roll not in self.Banner.rateups['5 star']:
          self.get5rated = True
        elif roll in self.Banner.list_chars['4 star'] and roll not in self.Banner.rateups['4 star']:
          self.get4rated = True
      else:
        results[roll] = results[roll] + 1
      
      simrolled += 1
      if simrolled % self.Banner.pitynum['5 star'] == 0:
        guarantee5 = True
      if simrolled % self.Banner.pitynum['4 star'] == 0:
        guarantee4 = True

    return results

  def runallsims(self):
    for i in tqdm(range(self.nsims)):
      single_sim = self._sim1()
      self.simresults.append(single_sim)

  def testroll(self):
    res = {}
    for i in range(10):
      roll = self._rollforstar('4 star')
      if roll not in res.keys():
        res[roll] = 1
      else:
        res[roll] = res[roll] + 1
    print(res)


In [ ]:
# Define a single roll

In [206]:
# Testing
b = Banner(all_banners['banner'][0])
sim = MonteCarloSimulator(b, nrolls=200, nsims=1000)

Loaded Banner: Venti
Characters on 5 star rateup: ['Venti'] at 50.0%
Characters on 4 star rateup: ['Barbara', 'Fischl', 'Xiangling'] at 50.0%


In [207]:
# print(sim.Banner.list_chars)
sim.testroll()

{'Chongyun': 1, 'Xiangling': 2, 'Beidou': 4, 'Sucrose': 1, 'Fischl': 2}


In [208]:
sim.runallsims()

In [215]:
# Turn sims into a pandas dataframe

simdf = pd.DataFrame(sim.simresults)
simdf = simdf.fillna(0)
simdf['Total Pulls'] = simdf.sum(axis=1)
simdf

,3 star,Venti,Xiangling,Beidou,Razor,Sucrose,Barbara,Chongyun,Keqing,Ninguang,Fischl,Diluc,Bennett,Jean,Xingqiu,Noelle,Qiqi,Mona,Total Pulls
0,168,2.0,8.0,4.0,1.0,1.0,3.0,1.0,2.0,1.0,5.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,200.0
1,165,2.0,2.0,2.0,1.0,3.0,6.0,1.0,1.0,3.0,5.0,0.0,3.0,1.0,3.0,1.0,1.0,0.0,200.0
2,174,2.0,6.0,0.0,1.0,1.0,3.0,3.0,0.0,2.0,4.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,200.0
3,170,2.0,2.0,0.0,1.0,3.0,10.0,2.0,0.0,1.0,1.0,0.0,4.0,1.0,1.0,1.0,1.0,0.0,200.0
4,171,1.0,5.0,1.0,1.0,2.0,5.0,1.0,1.0,5.0,3.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,172,2.0,4.0,3.0,1.0,0.0,2.0,2.0,0.0,1.0,7.0,0.0,1.0,0.0,1.0,3.0,1.0,0.0,200.0
996,170,1.0,3.0,2.0,2.0,0.0,7.0,2.0,1.0,1.0,3.0,0.0,0.0,0.0,5.0,2.0,1.0,0.0,200.0
997,166,2.0,5.0,1.0,4.0,1.0,7.0,1.0,0.0,2.0,4.0,1.0,1.0,0.0,3.0,1.0,0.0,1.0,200.0
998,166,1.0,8.0,4.0,3.0,0.0,5.0,0.0,1.0,2.0,4.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,200.0
